In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/DL4NLP/Homework_4

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/DL4NLP/Homework_4


In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
from transformers import GPT2Tokenizer, AutoTokenizer, GPT2LMHeadModel, BertTokenizer, Trainer, TrainingArguments, BertModel
from datasets import load_dataset
from collections import defaultdict

In [ ]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

## **Task 1: Byte pair encoding**

The goal is to calculate byte pair encodings for a given dataset. For a recap of the concept we refer to lecture 9, slides 34.ff and internet search.

### Task 1.1: Loading the Corpus
First we want to make sure we create a suffieciently large corpus. As dataset you should import the dataset "wikitext-2-raw-v1" from huggingface using the `load_dataset` function. The `corpus` should only contain the first 500 samples of the dataset. Additionally, you should filter out every empty entry in the corpus. Also print the first entry of the corpus.

Note: The dataset is a raw unprocessed dataset. Each sample in the dataset might consist of just a couple of words, a full sentence or multiple sentences. So the resulting corpus should look something like this: corpus = [ "couple of words", "This is one sentence.", "This is the first sentence. This is the second one.", .. ].

In [11]:
# Load WikiText-2 dataset "wikitext-2-raw-v1"
corpus = []
# YOUR CODE HERE
dataset = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")
corpus = [entry['text'] for entry in dataset['train'] if entry['text'].strip()]
corpus = corpus[:500]

In [12]:
corpus[0]

' = Valkyria Chronicles III = \n'

### Task 1.2 Word Frequencies + Init BPE
To preprocess the corpus we will first separate the text into words/tokens using gpt2's tokenizer and count the word frequencies in the corpus. Compute the number of occurences in the corpus for each word and store it in the word_freqs dictionary. This dictionary should then map each word to the number of occurences in the corpus.

Note: The tokenizer adds special characters to indicate the start or end of a word, such as 'Ġ'. You should NOT filter these special characters.

In [13]:
def compute_word_freqs(corpus):
  # Initialize a tokenizer from the transformers library
  tokenizer = AutoTokenizer.from_pretrained("gpt2")

  # Initialize a defaultdict to store word frequencies
  word_freqs = defaultdict(int)

  # We loop through to corups to calculate word frequencies
  for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    # --------------
    # YOUR CODE HERE
    # --------------
    for word in new_words:
      word_freqs[word] += 1


  return word_freqs

Run the cells below to initialize bpe. This will compute the word frequencies, generate the alphabet, generate the initial vocabulary (only consists of characters at the beginning), and generate the splits directory (maps each word to a list of its characters). These are important in the next steps to implement BPE.

In [14]:
def initialize_bpe(corpus):
  alphabet = []
  vocab = [""]
  splits = {}
  vocab_size = 200
  merges = {}

  word_freqs = compute_word_freqs(corpus)

  for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
  alphabet.sort()

  vocab = ["<|endoftext|>"] + alphabet.copy()
  splits = {word: [c for c in word] for word in word_freqs.keys()}
  return alphabet, vocab, splits, word_freqs

In [15]:
alphabet, vocab, splits, word_freqs = initialize_bpe(corpus)
print(f"Alpabet: \t{alphabet}\n\nInital Vocab: \t{vocab}\n\nSplits: \t{splits}\n\nWord Freq.: \t{word_freqs}")

Alpabet: 	['!', '"', '#', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '¢', '£', '¤', '¥', '¦', '©', 'ª', '«', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¸', '¹', '¼', '½', '¿', 'Â', 'Ã', 'Å', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ï', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'Ľ', 'ľ', 'Ŀ', 'Ł', 'ł', 'Ń']

Inital Vocab: 	['<|endoftext|>', '!', '"', '#', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 

### Task 1.3 Compute Pair Frequencies
Before merging the most frequent pairs of adjacent characters, we need to compute each pairs frequency. To do so, implement the function `compute_pair_frequency`. It takes as an input the splits dictionary from the previous task and returns the frequency of each character pair occuring in the corpus as a dictionary `pair_freqs`. This dictionary maps the character pair as a string to the number of occurences. For example for a corpus consisting only of the text "Hello hello!" the resulting dictionary would be {'He': 1, 'el': 2, 'll': 2, 'lo': 2, 'he': 1, 'o!': 1}.

In [16]:
# Define a function to compute the frequency of each pair of characters
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    # --------------
    # YOUR CODE HERE
    # --------------
    for _, value in splits.items():
      for i in range(len(value) - 1):
        if value[i] == ' ' or value[i+1] == ' ':
          continue
        pair_freqs[(value[i], value[i+1])] += 1
    return pair_freqs

# Initialize pair_freqs by calling the compute_pair_freqs function
pair_freqs = compute_pair_freqs(splits)

Run the following code to print the first 5 pair frequencies and the pair with the highest frequency.

In [17]:
print("First 5 pairs and frequencies:")
for i, key in enumerate(pair_freqs.keys()):
    print(f"'{key}': {pair_freqs[key]}")
    if i >= 4:
        break

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(f"\nBest pair: \n'{best_pair}': {max_freq}")

First 5 pairs and frequencies:
'('Ġ', '=')': 1
'('Ġ', 'V')': 41
'('V', 'a')': 15
'('a', 'l')': 442
'('l', 'k')': 12

Best pair: 
'('i', 'n')': 913


### Task 1.4 Merging Pairs
For BPE, we iteratively merge the most frequent pairs. To do so, you need to first implement the function `merge_pair`. It takes as an input two strings `a` and `b`, as well as the previously defined `splits` directory. It then merges  all occurences of the pair in this directory. For example:


```
splits = {'Hello': ['H', 'e' 'l', 'l', 'o'], 'low': ['l', 'o', 'w']}
new_splits = merge_pair('l', 'o', splits)
print(new_splits)
```
should return:
```
{'Hello': ['H', 'e' 'l', 'lo'], 'low': ['lo', 'w']}
```

Note:
For BPE, we do not just merge single characters. Hence, `merge_pair` should also be able to merge multiple characters, e.g. a='Hel' and b='lo' to 'Hello'.

In [18]:
# Define a function to merge the most frequent pair of characters
def merge_pair(a, b, splits):
  for word in word_freqs:
    split = splits[word]
    if len(split) == 1:
      continue
    # --------------
    # YOUR CODE HERE
    # --------------
    if a in split and b in split:
      i = 0
      new_split = []

      while i < len(split):
        if i == len(split) - 1:
          new_split.append(split[-1])
          break

        if a == split[i] and b == split[i+1]:
          new_split.append(f'{a}{b}')
          i += 2
        else:
          new_split.append(split[i])
          i += 1

      splits[word] = new_split
  return splits

Run the code below to merge the letters 'r' and 'e'.

In [19]:
new_splits = merge_pair('r', 'e', splits)
print(new_splits)

{'Ġ=': ['Ġ', '='], 'ĠValkyria': ['Ġ', 'V', 'a', 'l', 'k', 'y', 'r', 'i', 'a'], 'ĠChronicles': ['Ġ', 'C', 'h', 'r', 'o', 'n', 'i', 'c', 'l', 'e', 's'], 'ĠIII': ['Ġ', 'I', 'I', 'I'], 'ĠĊ': ['Ġ', 'Ċ'], 'ĠSenjÅį': ['Ġ', 'S', 'e', 'n', 'j', 'Å', 'į'], 'Ġno': ['Ġ', 'n', 'o'], 'Ġ3': ['Ġ', '3'], 'Ġ:': ['Ġ', ':'], 'ĠUnrecorded': ['Ġ', 'U', 'n', 're', 'c', 'o', 'r', 'd', 'e', 'd'], 'Ġ(': ['Ġ', '('], 'ĠJapanese': ['Ġ', 'J', 'a', 'p', 'a', 'n', 'e', 's', 'e'], 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': ['Ġ', 'æ', 'Ī', '¦', 'å', 'ł', '´', 'ã', 'ģ', '®', 'ã', 'ĥ', '´', 'ã', 'Ĥ', '¡', 'ã', 'ĥ', '«', 'ã', 'Ĥ', 'Ń', 'ã', 'ĥ', '¥', 'ã', 'ĥ', 'ª', 'ã', 'Ĥ', '¢'], '3': ['3'], 'Ġ,': ['Ġ', ','], 'Ġlit': ['Ġ', 'l', 'i', 't'], 'Ġ.': ['Ġ', '.'], 'Ġof': ['Ġ', 'o', 'f'], 'Ġthe': ['Ġ', 't', 'h', 'e'], 'ĠBattlefield': ['Ġ', 'B', 'a', 't', 't', 'l', 'e', 'f', 'i', 'e', 'l', 'd'], 'Ġ)': ['Ġ', ')'], 'Ġcommonly': ['Ġ', 'c', 'o', 'm', 'm', 'o', 'n', 'l', 'y'], 'Ġreferred': ['Ġ', 're', 'f', 'e', 'r', 're', 'd'], 'Ġto': ['Ġ', 't

### Task 1.5 Putting it all together - Implement BPE

Now we want to put all steps together and generate byte pair encodings. Implement the function `compute_bpe_vocab` that takes as an input the `corpus` and the `size` of the vocabulary that you want to generate. You will first need initialize the initial vocabulary, alphabet etc. as in task 1.2. You will then need to iteratively merge the most frequent character pairs in order to generate the new vocabulary until you have reached the desired `size`. To do so, you need to compute the pair frequencies as in task 1.3 and then merge the most frequent pairs as in 1.4. You can reuse all previously defined code for this task, e.g. the implemented functions from 1.3 and 1.4.

Hint: test your function by using a small vocabulary size, but keep in mind that the initial vocabulary size is already >100.

In [20]:
def compute_bpe_vocab(corpus, size):
  # --------------
  # YOUR CODE HERE
  # --------------

  alphabet, vocab, splits, word_freqs = initialize_bpe(corpus)

  while True:
    pair_freqs = compute_pair_freqs(splits=splits)
    best_pair = max(pair_freqs, key=pair_freqs.get)

    splits = merge_pair(best_pair[0], best_pair[1], splits)
    vocab.append(f'{best_pair[0]}{best_pair[1]}')

    if len(vocab) == size:
      break

  return vocab


### Task 1.6 Compute different Vocab Sizes
Compute the BPE vocabulary for the initialized corpus from task 1.1 using a size of 1000 and 3000. Print the last 20 entries of each generated vocabulary. What do you notice while computing and in both of their outputs.

In [21]:
# YOUR CODE FOR SIZE=1000
vocab_1000 = compute_bpe_vocab(corpus=corpus, size=1000)
vocab_1000[-20:]

['Ġcle',
 'Ġmyth',
 'Ġsymb',
 'Ġsymbol',
 'ky',
 'II',
 'ield',
 'Ġrefer',
 'velop',
 'eased',
 'Ġtim',
 'uns',
 'eless',
 'Ġmil',
 'uring',
 'Ġperform',
 'Ġunder',
 'Ġmulti',
 'aking',
 'iving']

In [22]:
# YOUR CODE FOR SIZE=3000
vocab_3000 = compute_bpe_vocab(corpus=corpus, size=3000)
vocab_3000[-20:]

['teen',
 'rill',
 'kin',
 'ĠCre',
 'ĠND',
 'ĠSy',
 'Ġimport',
 'Ġmob',
 'Ġconnec',
 'outed',
 'ĠAtl',
 'Ġcasem',
 'ĠScot',
 'Ġblockad',
 'orting',
 'chant',
 'de',
 'ulled',
 'Ġsteam',
 'Ġtub']

**YOUR ANSWER:**



The computation time for the larger vocabulary size takes significantly longer. Taking a look at the last pairs of the smaller vocabulary, one can see that there mostly parts using in a lot of words like 'aking' and 'iving' and even complete words like 'perform' or 'symbol'. When taking a look at the last pairs pairs of the larger vocabulary, there are mostly smaller parts that are probably used in less common words, like 'ND', 'Sy', etc.

# **Task 2: Getting to Know Foundation Models**

In this task, we want to load two different versions of GPT-2 and compare their performance.

### **Task 2.1 GPT2 small**

Load the **GPT2LMHeadModel** and **GPT2Tokenizer** from the HuggingFace **transformers** library. Initialize both with the pre-trained 'gpt2' configuration. \\
Print a single GPT-2 block from the model's 10th layer. \\
Calculate and *print* the total number of parameters in the GPT-2 model.

In [23]:
# YOUR CODE HERE

model_name = 'gpt2'

model_small = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer_small = GPT2Tokenizer.from_pretrained(model_name)

layer_10_block = model_small.transformer.h[9]  # Accessing the 10th layer (index 9)
print("GPT-2 Block from the 10th layer:", layer_10_block)

def count_parameters(model):
  #YOUR CODE HERE
  return sum(p.numel() for p in model.parameters())

# GPT2_small parameter trainable parameters:
print(f"The model has {count_parameters(model_small):,} trainable parameters")

GPT-2 Block from the 10th layer: GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
The model has 124,439,808 trainable parameters


### **Task 2.2 GPT2 large**
Because we want to compare two versions of GPT, you now also load the gpt2-large from huggingface as model. \\
Import the dataset "wikitext-2-raw-v1" from Hugging Face as an evaluation dataset. Make sure to use the 200 first entries of the test dataset. \\
Lastly, print the parameters of the gpt2-large model.

In [24]:
# YOUR CODE HERE
model_name = "gpt2-large"

model_large = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer_large = GPT2Tokenizer.from_pretrained(model_name)

print(f"The model has {count_parameters(model_large):,} trainable parameters")

The model has 774,030,080 trainable parameters


In [25]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
test_data = dataset['test'][:200]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

### **Task 2.3 Perplexity**
As a next step, we want to compute the perplexity score of both models. Remember the perplexity function from lecture 5 slide 20. Write a function that takes a model, tokenizer, and a text string as inputs and returns the perplexity score for the given text.

Hint: you should encode the text, generate outputs for inputs and corresponding labels and then calculate perplexity.



In [26]:
def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text)
    input_ids = torch.tensor(inputs["input_ids"])

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)

    loss = outputs.loss

    perplexity = 2**loss
    return perplexity

In [27]:
import numpy as np

perplexities_small = []
perplexities_large = []
# Loop over the dataset, append scores to the lists
# YOUR CODE HERE
for text in test_data['text']:
  if text:
    perplexities_small.append(calculate_perplexity(
      model=model_small,
      tokenizer=tokenizer_small,
      text=text
    ))

    perplexities_large.append(calculate_perplexity(
      model=model_large,
      tokenizer=tokenizer_large,
      text=text
    ))

# Calculate and print average perplexities
print("Average perplexity (GPT2 ): ", np.nanmean(perplexities_small))
print("Average perplexity (GPT2 large): ", np.nanmean(perplexities_large))

Average perplexity (GPT2 ):  56.212948
Average perplexity (GPT2 large):  47.954422


Explain the calculated perplexities for both gpt2 models in 2-3 sentences.

As the perplexity is simply 2 to the power of the loss, a larger loss leads to a higher perplexity. As GPT2 Large has been trained on more data and has more parameters, it will likely produce results that are closer to the labels. Therefore the model has a lower loss and therefore a lower perplexity than the smaller model.

### **Task 2.4 Masking attention**
**1)** What is an attention mask? When and why is it usually used?

An attention mask is a binary array which indicates which tokens should be attented to or ignored.  
They are useful when trying to ignore irrelevant information.  
They are used in a lot of use cases, one of them are autoregressive models like gpt-2.  
As these models only generate a token at a time and are not supposed to attend to the future tokens, attention masks are often used to mask out future tokens.

**2)** Your task is now to run a prompt in regular inference, but with an attention mask. You can reuse the gpt2_small model. Follow the steps below.

In [28]:
text_prompt = "The sun was setting behind"
mask_start = 3  # Start index of the region to mask
mask_end = 5  # End index of the region to mask

# Step 1: Implement attention masking
def get_attention_mask(input_ids, mask_start_idx, mask_end_idx):
    # -----------------------
    # YOUR CODE HERE
    # -----------------------

    attention_mask = torch.ones_like(input_ids)
    attention_mask[:, mask_start_idx:mask_end_idx] = 0
    return attention_mask

# Step 2: Apply attention masks
input_ids = tokenizer_small.encode(text_prompt, return_tensors='pt')
attention_mask = get_attention_mask(input_ids, mask_start, mask_end)

# Step 3: Generate output using modified attention mechanism
with torch.no_grad():
    outputs = model_small.generate(input_ids, attention_mask=attention_mask)
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("Text Prompt:", text_prompt)
print("Generated Text:", generated_text)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Text Prompt: The sun was setting behind
Generated Text: The sun was setting behind the only thing that could stop him.

"I'm sorry,


In [29]:
# Step 5: Run the same prompt without attention mask

# YOUR CODE HERE
text_prompt = "The sun was setting behind"

input_ids = tokenizer_small.encode(text_prompt, return_tensors='pt')
attention_mask = get_attention_mask(input_ids, mask_start, mask_end)

with torch.no_grad():
    outputs = model_small.generate(input_ids, attention_mask=torch.ones_like(input_ids))
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

print("Text Prompt:", text_prompt)
print("Generated Text:", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text Prompt: The sun was setting behind
Generated Text: The sun was setting behind the clouds, and the wind was blowing in the direction of the sun.


**Step 6:** Compare the two outputs and describe your findings in upto two sentences.

Without an attention mask, the whole input sequence is used to generate the next token and therefore in this case produces a sentence that makes a lot more sense than the first one.

### **Task 2.5 Fine tuning BERT**

In the lecture you have discovered methods to fine tune BERT for various different tasks. In this exercise you will look into the sentiment analysis task with BERT.

We will work with the [Stanford Sentiment Treebank](https://huggingface.co/datasets/stanfordnlp/sst2) dataset, which contains movie reviews with binary sentiment labels (positive or negative).

**a)** What are [CLS] tokens and why are they important for classification tasks?

The [CLS] token is a special token added to the beginning of every input sequence in BERT and other transformer models, serving as a representative summary of the entire sequence.
Its output vector is used for classification tasks because it captures the contextual information of the whole input.

Run this code to load the dataset and tokenize it.

In [30]:
seed_everything(seed=42)

dataset = load_dataset('glue', 'sst2')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def sample_dataset(dataset, fraction=0.1):
    return dataset.shuffle(seed=42).select(range(int(len(dataset) * fraction)))

train_dataset = sample_dataset(dataset['train'], fraction=0.1)
val_dataset = sample_dataset(dataset['validation'], fraction=0.1)

def tokenize(sample):
  return tokenizer(sample['sentence'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.remove_columns(['sentence'])
val_dataset = val_dataset.remove_columns(['sentence'])
train_dataset.set_format('torch')
val_dataset.set_format('torch')

print(len(train_dataset))


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/6734 [00:00<?, ? examples/s]

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

6734


b) We will use [`pytorch.nn`](https://pytorch.org/docs/stable/nn.html) to build our classification model. **Implement a class** `BertSentClassification` inheriting [`pytorch.nn.Module`](https://pytorch.org/docs/stable/nn.html). \\
**Hint:** Remember how we can use the CLS token for classification tasks.

In [31]:

import torch.nn as nn

class BertSentClassification(nn.Module):
  def __init__(self, bert, num_labels):
    # YOUR CODE HERE
    super(BertSentClassification, self).__init__()
    self.bert = bert
    self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

  def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
    logits = None
    # YOUR CODE HERE
    # You should save the result of the classifier in the parameter "logits", so that the remaining code works as intended
    # ---------------------

    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask , token_type_ids=token_type_ids)
    cls_output = outputs[1]
    logits = self.classifier(cls_output)

    # ---------------------
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits, labels)

    return (loss, logits) if loss is not None else logits

**c)** **Initialize the model** of this class. Use the pretrained [uncased BERT](https://huggingface.co/google-bert/bert-base-uncased) for training.

In [32]:
from transformers import BertModel
# YOUR CODE HERE
model = BertSentClassification(
    bert=BertModel.from_pretrained("bert-base-uncased"),
    num_labels=2
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Run this code to train the model. Training takes time, so please consider changing the hardware accelerator to T4 GPU in the settings (Runtime -> Change runtime type -> Hardware accelerator).

In [33]:
device = "cuda"
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.321826
2,0.345700,0.439531
3,0.119200,0.593406


TrainOutput(global_step=1263, training_loss=0.19429165393892772, metrics={'train_runtime': 620.7881, 'train_samples_per_second': 32.543, 'train_steps_per_second': 2.035, 'total_flos': 0.0, 'train_loss': 0.19429165393892772, 'epoch': 3.0})

**d)** Print the **accuracy** and **F1-score** on the validation dataset. You can use the trainer to get the predictions and true labels.

In [34]:
from sklearn.metrics import accuracy_score, f1_score

In [35]:
# YOUR CODE HERE
eval_result = trainer.predict(
    val_dataset
)

print(f"Accuracy: {accuracy_score(eval_result.label_ids, np.argmax(eval_result.predictions, axis=1)):.4f}")
print(f"F1-Score: {f1_score(eval_result.label_ids, np.argmax(eval_result.predictions, axis=1)):.4f}")

Accuracy: 0.8851
F1-Score: 0.9000
